In [1]:
import pandas as pd

In [2]:
from iac_sketch import data, parse, sketch

In [68]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(parse)
importlib.reload(sketch)
architect = sketch.Architect("../public/components")
registry = architect.parse()

In [69]:
links = registry["links"]
links

,entity,comp_ind,links,link_type
0,request_workflow,1,accept_request --> build_report\nbuild_report ...,dependency
1,build_report,2,define_report --> set_up_environment\nset_up_e...,dependency
2,run_report,2,run_etl --> run_cohort_notebook\nrun_cohort_no...,dependency


In [70]:
# Parse the links
exploded_links = (
    links["links"]
    # Split on newlines
    .str.strip()
    .str.split("\n")
    .explode()
    # Split on arrows
    .str.strip()
    .str.split("-->", expand=True)
    # Rename columns
    .rename(columns={0: "source", 1: "target"})
)
# Add the parsed results back to the original DataFrame
links = links.join(exploded_links).drop(columns=["links"]).reset_index(drop=True)
links

,entity,comp_ind,link_type,source,target
0,request_workflow,1,dependency,accept_request,build_report
1,request_workflow,1,dependency,build_report,deliver_report
2,build_report,2,dependency,define_report,set_up_environment
3,build_report,2,dependency,set_up_environment,set_up_report
4,build_report,2,dependency,set_up_report,work_on_report
5,build_report,2,dependency,work_on_report,get_feedback_on_report
6,build_report,2,dependency,get_feedback_on_report,work_on_report
7,build_report,2,dependency,get_feedback_on_report,finalize_report
8,run_report,2,dependency,run_etl,run_cohort_notebook
9,run_report,2,dependency,run_cohort_notebook,run_report_notebook


In [85]:
registry.entities.at[0, "component_entity"] = "algorithm"

In [83]:
entities_by_group = registry.entities.groupby("component_entity")
entities_by_group

In [91]:
entities_by_group

In [ ]:
# Get the new comp index, using the metadata
links["comp_ind"] = links.groupby("entity").cumcount()
links["comp_ind"] += links.merge(registry["metadata"], on="entity", how="left")["n_comps"]


In [ ]:
(
    links["comp_ind"]
    +
)

0    3
1    4
2    4
3    5
4    6
5    7
6    8
7    9
8    4
9    5
dtype: int64

In [8]:
# Find invalid requirements
reqs = registry.view(["requirement", "status", "task", "input"])
is_valid = reqs["status"].isin(["closed", "removed"])
invalid_reqs = reqs.loc[~is_valid]
invalid_reqs

,entity,status,requester [person],input
0,accept_request,in design,The person seeking to submit a request.,NaN
1,deliver_report,new,NaN,NaN
2,archive_data_products,new,NaN,NaN
3,framework_manifest_data,in design,NaN,NaN
4,framework_manifest_documentation,new,NaN,NaN
5,power_users,new,NaN,NaN
6,audit_cohorts,new,NaN,NaN


In [62]:
# Find invalid testcases
registry.view(["testcase", "status"])

,entity,status
0,basic_report_test,new
1,no_irb_test,new
2,updated_report_test,new
3,updated_report_new_analyst_test,new
